In [6]:
import pandas as pd
from matplotlib import pyplot as plt
from mendeleev.fetch import fetch_table
import numpy as np
from ipywidgets import widgets
import plotly.graph_objects as go


In [2]:
xrf_data = pd.read_excel("../data/interim/xrf_data_clean.xlsx")

In [36]:
standards_data = xrf_data[xrf_data["qaqc_type"]=="standard"]
lab_duplicates_data = xrf_data[xrf_data["qaqc_type"]=="lab duplicate"]
field_duplicates_data = xrf_data[xrf_data["qaqc_type"]=="field duplicate"]

In [37]:
standards = standards_data["sample_id"].unique()
lab_duplicates = lab_duplicates_data["sample_id"].unique()
field_duplicates = field_duplicates_data["sample_id"].unique()
dates = xrf_data["date"].unique()


In [38]:
standards_drift_data = standards_data
for standard in standards:
    for date in dates:
        initial_date = "2021-10-06"
        print(standard)
        if standards_drift_data[standards_drift_data["sample_id"]==standard and standards_drift_data["date"]==initial_date]: 
            initial_measurement = standards_drift_data[standards_drift_data["sample_id"]==standard and \
                                standards_drift_data["date"]==initial_date]
            measurement = standards_drift_data[standards_drift_data["sample_id"]==standard and \
                                standards_drift_data["date"]==date]

            standards_drift_data[standards_drift_data["sample_id"]==standard and standards_drift_data["date"]==date] = measurement - initial_measurement

oreas24b


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [39]:
plt.scatter(standards["date"],standards["Ca"])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [7]:
session = widgets.Dropdown(
    description='Session:',
    options=["All"] + xrf_data['date'].unique().tolist() 
)

group = widgets.Dropdown(
    description='Group: ',
    options=["All"] + xrf_data['group'].unique().tolist() 
)

sample_type = widgets.Dropdown(
    options=["All"] + xrf_data['sample_type'].unique().tolist(),
    description='Sample Type: '
)

qaqc_type = widgets.Dropdown(
    options=["All"] + xrf_data['qaqc_type'].unique().tolist(),
    description='QA/QC Type: '
)

In [8]:
# get elements analyzed
columns = xrf_data.columns.tolist()
ptable = fetch_table('elements').symbol.to_list()
elements = list(set(columns) & set(ptable))
elements.sort()


element = widgets.Dropdown(
    options=elements,
    description="Elements: "
)

In [15]:
g = go.FigureWidget()

for elem in elements: 
    g.add_trace(
        go.Histogram(x=xrf_data[elem], opacity=0.75, name=elem) 
    )

In [16]:
def validate(): 
    if (session.value     in xrf_data['date'].unique()        or session.value == "All")     and \
       (group.value       in xrf_data['group'].unique()       or group.value == "All")       and \
       (sample_type.value in xrf_data['sample_type'].unique() or sample_type.value == "All") and \
       (qaqc_type.value   in xrf_data['qaqc_type'].unique()   or qaqc_type.value == "All")   and \
       (element.value     in elements): 
        return True
    else:
        return False

In [17]:
def response(change):
    if validate(): # if is true
        visibilities = [False] * len(g.data)
        visible_index = elements.index(element.value)
        visibilities[visible_index] = True

        filter = dict.fromkeys(['session', 'group', 'sample_type', 'qaqc_type'])

        if session.value == "All": 
            filter['session'] = pd.Series([True] * xrf_data.shape[0])
        else:
            filter['session'] = pd.Series(xrf_data['date'] == session.value)

        if group.value == "All": 
            filter['group'] = pd.Series([True] * xrf_data.shape[0])
        else:
            filter['group'] = pd.Series(xrf_data['group'] == group.value)

        if sample_type.value == "All": 
            filter['sample_type'] = pd.Series([True] * xrf_data.shape[0])
        else:
            filter['sample_type'] = pd.Series(xrf_data['sample_type'] == sample_type.value) 

        if qaqc_type.value == "All": 
            filter['qaqc_type'] = pd.Series([True] * xrf_data.shape[0])
        else:
            filter['qaqc_type'] = pd.Series(xrf_data['qaqc_type'] == qaqc_type.value)                    

        filter_list = [session_bool and group_bool and sample_type_bool and qaqc_type_bool \
                       for session_bool, group_bool, sample_type_bool, qaqc_type_bool \
                       in zip(filter['session'], filter['group'], filter['sample_type'], filter['qaqc_type'])]
                        
        temp_df = xrf_data[filter_list]
        x =  [None] * len(g.data)
        for i, elem in enumerate(elements):
            x[i] = temp_df[elem]
        with g.batch_update(): # required to update data
            for i, (trace, visibility) in enumerate(zip(x, visibilities)): 
                g.data[i].x = x[i]
                g.data[i].visible = visibility


g.layout.xaxis.title = 'Concentration (ppm)'
g.layout.yaxis.title = 'No. of samples'


session.observe(response, names="value")
group.observe(response, names="value")
sample_type.observe(response, names="value")
qaqc_type.observe(response, names="value")
element.observe(response, names="value")


In [18]:
container1 = widgets.HBox([element, session, group])
container2 = widgets.HBox([sample_type, qaqc_type])
widgets.VBox([container1, container2, 
              g])